In [1]:
import pandas as pd


df_allseasons = pd.read_csv('cleaned_merged_seasons.csv')

df_allseasons.head()

/tmp/ipykernel_361783/2904350853.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_allseasons = pd.read_csv('cleaned_merged_seasons.csv')


,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,...,2.0,0.0,0,0,0,0,55,False,0,1
1,1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,...,1.0,0.0,1,0,0,0,60,True,0,1
2,2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,...,3.0,23.0,2,0,0,0,80,True,0,1
3,3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,...,1.0,0.0,0,0,0,0,50,False,0,1
4,4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,...,1.0,0.0,1,0,0,0,45,True,1,1


In [2]:
df_allseasons.drop(['Unnamed: 0', 'team_x'], axis=1, inplace=True)

In [3]:
df_allseasons.shape

(98402, 36)

In [22]:
df_allseasons.columns

Index(['season_x', 'name', 'position', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW', 'name_season', 'club_name'],
      dtype='object')

In [23]:
df_allseasons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98315 entries, 0 to 98401
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season_x           98315 non-null  object 
 1   name               98315 non-null  object 
 2   position           98315 non-null  object 
 3   assists            98315 non-null  int64  
 4   bonus              98315 non-null  int64  
 5   bps                98315 non-null  int64  
 6   clean_sheets       98315 non-null  int64  
 7   creativity         98315 non-null  float64
 8   element            98315 non-null  int64  
 9   fixture            98315 non-null  int64  
 10  goals_conceded     98315 non-null  int64  
 11  goals_scored       98315 non-null  int64  
 12  ict_index          98315 non-null  float64
 13  influence          98315 non-null  float64
 14  kickoff_time       98315 non-null  object 
 15  minutes            98315 non-null  int64  
 16  opponent_team      983

In [4]:
df_allseasons['name_season'] = df_allseasons['name'] + '_' + df_allseasons['season_x']

In [5]:
df_allseasons.name_season

0           Aaron Cresswell_2016-17
1              Aaron Lennon_2016-17
2              Aaron Ramsey_2016-17
3        Abdoulaye Doucouré_2016-17
4              Adam Forshaw_2016-17
                    ...            
98397         Wilfred Ndidi_2021-22
98398          Matt Ritchie_2021-22
98399        Nathan Redmond_2021-22
98400           Mathew Ryan_2021-22
98401       Ryan Fredericks_2021-22
Name: name_season, Length: 98402, dtype: object

In [24]:
df_allseasons.club_name

0        WHU
1        EVE
2        ARS
3        WAT
4        MID
        ... 
98397    LEI
98398    NEW
98399    SOU
98400    BHA
98401    WHU
Name: club_name, Length: 98315, dtype: object

In [6]:
df_allseasons.name.nunique()

989

In [8]:
import requests

# get yearly historic data from endpoint for available seasons and identify the keys in each disctionary using 2016 as an example
Y2016= requests.get('https://www.fantasynutmeg.com/api/history/season/2016-17').json()
Y2017= requests.get('https://www.fantasynutmeg.com/api/history/season/2017-18').json()
Y2018= requests.get('https://www.fantasynutmeg.com/api/history/season/2018-19').json()
Y2019= requests.get('https://www.fantasynutmeg.com/api/history/season/2019-20').json()
Y2020= requests.get('https://www.fantasynutmeg.com/api/history/season/2020-21').json()
Y2021= requests.get('https://www.fantasynutmeg.com/api/history/season/2021-22').json()
Y2022= requests.get('https://www.fantasynutmeg.com/api/history/season/2022-23').json()
Y2016.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [ ]:
# Hist from 16-21.
# Create a column with the First Name, Last Name and the Year.
# Combiine the name and the year of the cloned repo to match the hist data.
# Merge the two based on the First Name, Last Name and the Year. 

In [9]:
hist16_df = pd.DataFrame(Y2016['history'])
hist17_df = pd.DataFrame(Y2017['history'])
hist18_df = pd.DataFrame(Y2018['history'])
hist19_df = pd.DataFrame(Y2019['history'])
hist20_df = pd.DataFrame(Y2020['history'])
hist21_df = pd.DataFrame(Y2021['history'])

In [25]:
hist16_df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,year
0,0,0,18,100,100,0,48844,0,0,-3,...,2,3433,0,7905,0,0.0,0.4,Ospina,0,2016-17
1,0,2,660,100,100,12,11334,0,0,-1,...,134,432837,0,590727,0,0.0,24.8,Cech,2,2016-17
2,1,19,723,0,75,10,51507,0,0,1,...,121,965040,0,857277,0,0.0,19.8,Koscielny,4,2016-17
3,0,0,5,100,100,0,17127,0,0,-2,...,1,2141,0,5912,0,0.0,0.2,Mertesacker,0,2016-17
4,0,2,296,75,100,5,158074,0,0,-2,...,45,119600,0,100361,0,0.0,9.4,Gabriel,6,2016-17


In [10]:
hist16_df['year'] = hist16_df.apply(lambda x: "2016-17", axis=1)
hist17_df['year'] = hist17_df.apply(lambda x: "2017-18", axis=1)
hist18_df['year'] = hist18_df.apply(lambda x: "2018-19", axis=1)
hist19_df['year'] = hist19_df.apply(lambda x: "2019-20", axis=1)
hist20_df['year'] = hist20_df.apply(lambda x: "2020-21", axis=1)
hist21_df['year'] = hist21_df.apply(lambda x: "2021-22", axis=1)

In [11]:
hist_df = [hist16_df, hist17_df, hist18_df, hist19_df, hist20_df, hist21_df]

hist = pd.concat(hist_df, axis = 0, ignore_index=True)

In [12]:
hist.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,influence_rank,influence_rank_type,threat_rank,threat_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,0,0,18,100,100,0,48844,0,0,-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2,660,100,100,12,11334,0,0,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,19,723,0,75,10,51507,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,5,100,100,0,17127,0,0,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2,296,75,100,5,158074,0,0,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
hist['name_season'] = hist['first_name'] + ' ' + hist['second_name'] + '_' + hist['year']

In [14]:
hist.name_season

0                   David Ospina_2016-17
1                      Petr Cech_2016-17
2              Laurent Koscielny_2016-17
3                Per Mertesacker_2016-17
4       Gabriel Armando de Abreu_2016-17
                      ...               
4065                Adrian Blake_2021-22
4066                Jack Grieves_2021-22
4067              Joseph McGlynn_2021-22
4068             Anthony Mancini_2021-22
4069              Charlie Savage_2021-22
Name: name_season, Length: 4070, dtype: object

In [15]:
teams=dict(zip(hist.name_season, hist.team_name))

df_allseasons['club_name'] = df_allseasons['name_season'].map(teams)

In [16]:
subset  = ['Harry Kane_2017-18']
check = df_allseasons[df_allseasons.name_season.isin(subset)]
check

,season_x,name,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,name_season,club_name
8211,2017-18,Harry Kane,FWD,0,0,-6,1,30.6,394,10,...,1,0,0,0,125,False,1,1,Harry Kane_2017-18,TOT
8474,2017-18,Harry Kane,FWD,0,0,0,0,0.0,394,98,...,0,-621465,128901,750366,127,False,0,10,Harry Kane_2017-18,TOT
8746,2017-18,Harry Kane,FWD,0,0,6,1,13.3,394,108,...,2,543001,607390,64389,128,True,0,11,Harry Kane_2017-18,TOT
9020,2017-18,Harry Kane,FWD,0,0,1,0,13.8,394,111,...,1,-10246,84353,94599,128,False,1,12,Harry Kane_2017-18,TOT
9294,2017-18,Harry Kane,FWD,0,2,25,0,6.8,394,128,...,8,-17553,59877,77430,128,True,0,13,Harry Kane_2017-18,TOT
9569,2017-18,Harry Kane,FWD,0,0,26,0,27.2,394,132,...,6,-7422,51022,58444,128,False,0,14,Harry Kane_2017-18,TOT
9845,2017-18,Harry Kane,FWD,0,0,3,0,15.9,394,149,...,2,13608,59740,46132,128,False,0,15,Harry Kane_2017-18,TOT
10122,2017-18,Harry Kane,FWD,0,2,37,0,3.9,394,158,...,12,-36174,69065,105239,128,True,0,16,Harry Kane_2017-18,TOT
10399,2017-18,Harry Kane,FWD,0,0,11,1,32.4,394,169,...,2,204980,213053,8073,129,True,0,17,Harry Kane_2017-18,TOT
10678,2017-18,Harry Kane,FWD,0,0,-2,0,4.8,394,177,...,1,-93532,41784,135316,129,False,1,18,Harry Kane_2017-18,TOT


In [18]:
df_allseasons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98402 entries, 0 to 98401
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season_x           98402 non-null  object 
 1   name               98402 non-null  object 
 2   position           98402 non-null  object 
 3   assists            98402 non-null  int64  
 4   bonus              98402 non-null  int64  
 5   bps                98402 non-null  int64  
 6   clean_sheets       98402 non-null  int64  
 7   creativity         98402 non-null  float64
 8   element            98402 non-null  int64  
 9   fixture            98402 non-null  int64  
 10  goals_conceded     98402 non-null  int64  
 11  goals_scored       98402 non-null  int64  
 12  ict_index          98402 non-null  float64
 13  influence          98402 non-null  float64
 14  kickoff_time       98402 non-null  object 
 15  minutes            98402 non-null  int64  
 16  opponent_team      984

In [26]:
df_allseasons.club_name.info()

<class 'pandas.core.series.Series'>
Int64Index: 98315 entries, 0 to 98401
Series name: club_name
Non-Null Count  Dtype 
--------------  ----- 
98315 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


In [19]:
df_allseasons.dropna(inplace=True)

In [20]:
df_allseasons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98315 entries, 0 to 98401
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season_x           98315 non-null  object 
 1   name               98315 non-null  object 
 2   position           98315 non-null  object 
 3   assists            98315 non-null  int64  
 4   bonus              98315 non-null  int64  
 5   bps                98315 non-null  int64  
 6   clean_sheets       98315 non-null  int64  
 7   creativity         98315 non-null  float64
 8   element            98315 non-null  int64  
 9   fixture            98315 non-null  int64  
 10  goals_conceded     98315 non-null  int64  
 11  goals_scored       98315 non-null  int64  
 12  ict_index          98315 non-null  float64
 13  influence          98315 non-null  float64
 14  kickoff_time       98315 non-null  object 
 15  minutes            98315 non-null  int64  
 16  opponent_team      983

In [27]:
df_allseasons.dropna(how='all')

,season_x,name,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,name_season,club_name
0,2016-17,Aaron Cresswell,DEF,0,0,0,0,0.0,454,10,...,0,0,0,0,55,False,0,1,Aaron Cresswell_2016-17,WHU
1,2016-17,Aaron Lennon,MID,0,0,6,0,0.3,142,3,...,1,0,0,0,60,True,0,1,Aaron Lennon_2016-17,EVE
2,2016-17,Aaron Ramsey,MID,0,0,5,0,4.9,16,8,...,2,0,0,0,80,True,0,1,Aaron Ramsey_2016-17,ARS
3,2016-17,Abdoulaye Doucouré,MID,0,0,0,0,0.0,482,7,...,0,0,0,0,50,False,0,1,Abdoulaye Doucouré_2016-17,WAT
4,2016-17,Adam Forshaw,MID,0,0,3,0,1.3,286,6,...,1,0,0,0,45,True,1,1,Adam Forshaw_2016-17,MID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98397,2021-22,Wilfred Ndidi,MID,0,0,0,0,0.0,216,377,...,0,-202,22,224,48,True,0,38,Wilfred Ndidi_2021-22,LEI
98398,2021-22,Matt Ritchie,DEF,0,0,3,0,0.0,292,374,...,1,143,396,253,49,False,0,38,Matt Ritchie_2021-22,NEW
98399,2021-22,Nathan Redmond,MID,0,0,5,0,0.0,336,377,...,2,455,683,228,59,False,0,38,Nathan Redmond_2021-22,SOU
98400,2021-22,Mathew Ryan,GK,0,0,0,0,0.0,65,373,...,0,-2,0,2,45,True,0,38,Mathew Ryan_2021-22,BHA


In [ ]:
#get current season data from FPL API endpoints and identify the keys
fpl_base_url = 'https://fantasy.premierleague.com/api/'
current_season = requests.get(fpl_base_url+'bootstrap-static/').json()
#json = r.json()
current_season.keys()

In [ ]:
#create dataframes for the current season dictionary keys for data exploration
#- Contains summary of Gameweek data
events_df = pd.DataFrame(current_season['events']) #
phases_df = pd.DataFrame(current_season['phases']) #Shows calendar months for game weeks
teams_df = pd.DataFrame(current_season['teams'])
players_df = pd.DataFrame(current_season['elements'])
element_stats_df = pd.DataFrame(current_season['element_stats'])
element_types_df = pd.DataFrame(current_season['element_types'])

In [ ]:
#get current season fixtures from FPL API endpoint and create Dataframe

current_season_fixtures = requests.get(fpl_base_url+'fixtures/').json()
fixtures_df = pd.DataFrame(current_season_fixtures)
fixtures_df.head()